In [0]:
%run /Workspace/carsdataflow_databricks_notebooks/setup

# Create Fact Table

In [0]:
silver_path = "abfss://silver@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/"
gold_path = "abfss://gold@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/"

In [0]:
%python
# Reading the data from silver container

df = spark.read.format("Delta").load(silver_path)
display(df)

In [0]:
# brining all the dimension into the df

dim_model_df = spark.sql("select * from carsdataflow_silver.dim_model")
dim_branch_df = spark.sql("select * from carsdataflow_silver.dim_branch")
dim_dealer_df = spark.sql("select * from carsdataflow_silver.dim_dealer")
dim_date_df = spark.sql("select * from carsdataflow_silver.dim_date")

dim_model_df.display()
dim_branch_df.display()
dim_dealer_df.display()
dim_date_df.display()


In [0]:
# Bringing dim_keys to the Fact Table

fact_table = df.join(dim_model_df, df.Model_ID == dim_model_df.Model_ID, "left")\
                .join(dim_branch_df, df.Branch_ID == dim_branch_df.Branch_ID, "left")\
                    .join(dim_dealer_df, df.Dealer_ID == dim_dealer_df.Dealer_ID, "left")\
                        .join(dim_date_df, df.Date_ID == dim_date_df.Date_ID, "left")\
                            .select(df.Revenue, df.Units_Sold, df.Units_Sold_Per_Unit, dim_model_df.Model_Key, dim_branch_df.Branch_Key, dim_dealer_df.Dealer_Key, dim_date_df.Date_Key)

display(fact_table)

### Writing Fact Table to Gold Container

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("carsdataflow_gold.fact_table"):
    delta_table = DeltaTable.forName(spark, "carsdataflow_gold.fact_table")
    delta_table.alias("t").merge(fact_table.alias("s"), "t.Date_Key = s.Date_Key AND t.Branch_Key = s.Branch_Key AND t.Dealer_Key = s.Dealer_Key AND t.Model_Key = s.Model_Key")

else:
    fact_table.write.format("delta").mode("overwrite").option("path", gold_path + "fact_table").saveAsTable("carsdataflow_gold.fact_table")


In [0]:
%sql
select * from carsdataflow_gold.fact_table